In [7]:
import numpy as np
import pandas as pd
import antropy as ent
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.fft import fft, fftfreq 
import seaborn as sns
from scipy import signal
import seaborn as sns
from scipy import stats
from os.path import exists

sns.set_context('paper')

# Definiendo las funciones requeridas

In [20]:
# La transformada de Fourier
def getFourierTransform(data, measfreq, maxfreq):
    data_fft = 10*np.log10(np.abs(fft(data))**2)
    freqs = fftfreq(len(data_fft), 1/measfreq)
    mask = (freqs > 0) & (freqs < maxfreq)
    return freqs[mask],data_fft[mask]

# Permutation entropy
def get_perm_entropy(data, metric, measfreq, dtime, overlap):
    timewindow = dtime/(1-overlap)
    ndata = int(timewindow*measfreq)
    step = int(measfreq*dtime)
    entropy = []
    times = []
    for i in np.arange(np.floor((len(data)-ndata)/step).astype(int)+1):
        times.append((i*step+ndata/2)/measfreq)
        entropy.append(ent.perm_entropy(data[i*step:i*step+ndata], order=metric, normalize = True))
    return np.array(times), np.array(entropy)

# Sample entropy
def get_samp_entropy(data, metric, measfreq, dtime, overlap):
    timewindow = dtime/(1-overlap)
    ndata = int(timewindow*measfreq)
    step = int(measfreq*dtime)
    entropy = []
    times = []
    for i in np.arange(np.floor((len(data)-ndata)/step).astype(int)+1):
        times.append((i*step+ndata/2)/measfreq)
        entropy.append(ent.sample_entropy(data[i*step:i*step+ndata], order=metric))
    return np.array(times), np.array(entropy)

# MultiScale Entropy
def get_ms_entropy(data, metric, scale, measfreq, dtime, overlap):
    # the time window and the step are scaled according to scale, the overlap is mantained
    timewindow = scale*dtime/(1-overlap)  # the length of the time window
    ndata = int(timewindow*measfreq)  # the number of data in each time window
    step = int(measfreq*dtime*scale)  # the number of data in each step
    entropy = []
    times = []
    nndata = np.floor(ndata/scale).astype(int)
  #  if (nndata < 100):
#        print("Warning! The entropy is calculated with fewer than 100 points %d"%nndata)
    for i in np.arange(np.floor((len(data)-ndata)/step).astype(int)+1):
        newdata = data[i*step:i*step+ndata] # the new data
        msdata = []
        for j in np.arange(nndata):
            msdata.append(newdata[j*scale:(j+1)*scale].mean()) # the average of the data
        times.append((i*step+ndata/2)/measfreq) # the time average of the interval
        entropy.append(ent.sample_entropy(np.array(msdata), order=metric)) # The calculation of the multiscale entropy
    return np.array(times), np.array(entropy)

# Figura espectral por bandas
def createSpectralFigureBands(subject, data, Nsesion, sesiones, kind, eeg, metric, measfreq, maxfreq, dtime, overlap):
    # primero calculamos la FFT
    freqs_fft, data_fft = getFourierTransform(data[Nsesion][kind][eeg].values, measfreq, maxfreq)
    # definimos las bandas
    bands = [4,8,12,35]
    bandnames = ['delta', 'theta', 'alpha', 'beta', 'gamma']
    
    perments = []
    sampents = []
    
    for i in range(len(bandnames)):
        if i == 0: # first band
            sos = signal.butter(8,bands[i], btype = 'lowpass', output = 'sos', fs = 240)
            filtered = signal.sosfilt(sos, data[Nsesion][kind][eeg])
            perments.append(get_perm_entropy(filtered, metric, measfreq, dtime, overlap))
            sampents.append(get_samp_entropy(filtered, metric, measfreq, dtime, overlap))
        elif i == (len(bandnames)-1): # last band
            sos = signal.butter(8,bands[i-1], btype = 'highpass', output = 'sos', fs = 240)
            filtered = signal.sosfilt(sos, data[Nsesion][kind][eeg])
            perments.append(get_perm_entropy(filtered, metric, measfreq, dtime, overlap))
            sampents.append(get_samp_entropy(filtered, metric, measfreq, dtime, overlap))
        else: # middle bands
            sos = signal.butter(8,[bands[i-1],bands[i]], btype = 'bandpass', output = 'sos', fs = 240)
            filtered = signal.sosfilt(sos, data[Nsesion][kind][eeg])
            perments.append(get_perm_entropy(filtered, metric, measfreq, dtime, overlap))
            sampents.append(get_samp_entropy(filtered, metric, measfreq, dtime, overlap))
            
    
    # The figure parameters
    # individual width
    wi=3.5
    # individual height
    hi=2.0
    # left margin
    lm=0.72
    # right margin
    rm=0.1
    # bottom margin
    bm=0.45
    # top margin
    tm=0.7
    # horizontal gap
    hg=0.3
    # vertical gap
    vg=0.5
    # number of columns
    nc=1
    # number of rows
    nr=5

    # The calculations
    sx=wi*nc+lm+rm+hg*(nc-1)  # Figure x
    sy=hi*nr+bm+tm+vg*(nr-1)  # Figure y
    t=(sy-tm)/sy
    b=bm/sy
    l=lm/sx
    r=(sx-rm)/sx
    hr=vg/hi
    wr=hg/wi
    
    # ahora creamos la gráfica de los tres elementos
    fig, ax = plt.subplots(nr,nc,figsize=(sx,sy))
    fig.suptitle('Estudio espectral:\n '+subject+' '+str(sesiones[Nsesion])+', '+kind+', señal '+eeg, fontsize=16)
    # primero ponemos los datos originales
    ax[0].plot(np.arange(len(data[Nsesion][kind][eeg]))/measfreq,data[Nsesion][kind][eeg])
    ax[0].set_xlabel('tiempo (s)')
    ax[0].set_ylabel('señal EEG')
    # posteriormente ponemos el spectrograma
    #ax[2].imshow(np.transpose(spec)[::-1,:], aspect='auto', extent = [times[0],times[-1],freqs_spec[0],freqs_spec[-1]])
    window = dtime/(1-overlap)
    aux,aux,aux,spc=ax[3].specgram(data[Nsesion][kind][eeg], Fs=measfreq, scale = 'dB', NFFT = int(measfreq*window), noverlap = int((window-dtime)*measfreq), mode = 'magnitude', cmap = 'jet')
    ax[0].set_xlim(ax[3].get_xlim())
    ax[3].set_xlabel('tiempo (s)')
    ax[3].set_ylabel('frecuencia (Hz)')
    # finalmente, la transformada de fourier completa
    ax[4].plot(freqs_fft,data_fft)
    ax[4].set_xlabel('frecuencia (Hz)')
    ax[4].set_ylabel('potencia (dB)')
    ax[4].set_xlim(0,maxfreq)
    # Ahora la entropía de permutación
    for i in range(len(bandnames)):
        ax[1].plot(perments[i][0],perments[i][1], label= 'perm_ent_'+bandnames[i], color='C%d'%i)
        ax[1].axhline(y=perments[i][1].mean(), color='C%d'%i, linestyle='--')
    ax[1].set_xlabel('tiempo (s)')
    ax[1].set_ylabel('entropía')
    ax[1].set_ylim(0,1)
    ax[1].set_xlim(ax[3].get_xlim())
    ax[1].legend(fontsize = 6)
    # ahora la entropía de la muestra
    for i in range(len(bandnames)):
        ax[2].plot(sampents[i][0],sampents[i][1], label= 'samp_ent_'+bandnames[i], color='C%d'%i)
        ax[2].axhline(y=sampents[i][1].mean(), color='C%d'%i, linestyle='--')
    ax[2].set_xlabel('tiempo (s)')
    ax[2].set_ylabel('entropía')
    ax[2].set_ylim(0,1)
    ax[2].set_xlim(ax[3].get_xlim())
    ax[2].legend(fontsize = 6)
    # The adjustment
    plt.subplots_adjust(wspace=wr,hspace=hr,bottom=b, top=t, left=l, right=r)
    plt.savefig('images/sp_'+subject+'_%d_'%(sesiones[Nsesion])+kind+'_'+eeg+'.png', dpi = 200)
    plt.close()


# Cálculo general de entropía, Control
def getEntropyAnalysis(subject, phases, data, ent_kind, sesiones, metric, scale, measfreq, dtime, overlap):
    # data, the list with all the data
    # ent_kind, a string: either 'sampen', 'permen' or 'msen'
    if ent_kind not in ['sampen', 'permen', 'msen']:
        print("The kind of entropy is not correct")
        return 
    # metric, the metric to calculate the entropy
    # scale, needed for 'msen' the scale for multiscale entropy, not used for sampen or permen
    # measfreq, the frequency of the measurement
    # dtime, the interval step
    # overlap, the interval overlap. The interval size is calculated as dtime/(1-overlap)
    
    # we first check whether the entropy has already been calculated
    if ent_kind == 'msen':
        fileroot = 'results/entropy_'+subject+'_'+ent_kind+'_'+str(metric)+'_'+str(scale)+'_'+str(dtime)+'_'+str(overlap)
    else:
        fileroot = 'results/entropy_'+subject+'_'+ent_kind+'_'+str(metric)+'_'+str(dtime)+'_'+str(overlap)
    
    comparison = phases[0]+phases[1]
    if exists(fileroot+'_'+comparison+'_stats.xlsx'):  # the file exists and we can proceed
        return
    else:
        # we define a dataframe with the entropy calculations
        ent_data_band = pd.DataFrame()

        # now we define lists to store the information
        ents = []
        eegs = []
        bands = []
        sesions = []
        timing = []
        bands = [4,8,12,35]
        bandnames = ['delta', 'theta', 'alpha', 'beta', 'gamma', 'all bands']
        banddata = []

        for i,ses  in enumerate(sesiones):
            for key in phases:
                for eeg in ['EEG1','EEG2']:
                    for k in range(len(bandnames)):
                        try:
                            if k == 0: # first band
                                sos = signal.butter(8, bands[k], btype = 'lowpass', output = 'sos', fs = measfreq)
                                filtered = signal.sosfilt(sos, data[i][key][eeg])
                                if ent_kind == 'sampen':
                                    times, ent = get_samp_entropy(filtered, metric, measfreq, dtime, overlap)
                                elif ent_kind == 'permen':
                                    times, ent = get_perm_entropy(filtered, metric, measfreq, dtime, overlap)
                                elif ent_kind == 'msen':
                                    times, ent = get_ms_entropy(filtered, metric, scale, measfreq, dtime, overlap)

                            elif k == len(bandnames)-1: # all bands
                                if ent_kind == 'sampen':
                                    times, ent = get_samp_entropy(data[i][key][eeg], metric, measfreq, dtime, overlap)
                                elif ent_kind == 'permen':
                                    times, ent = get_perm_entropy(data[i][key][eeg], metric, measfreq, dtime, overlap)
                                elif ent_kind == 'msen':
                                    times, ent = get_ms_entropy(data[i][key][eeg], metric, scale, measfreq, dtime, overlap)

                            elif k == (len(bandnames)-2): # last band
                                sos = signal.butter(8,bands[k-1], btype = 'highpass', output = 'sos', fs = measfreq)
                                filtered = signal.sosfilt(sos, data[i][key][eeg])
                                if ent_kind == 'sampen':
                                    times, ent = get_samp_entropy(filtered, metric, measfreq, dtime, overlap)
                                elif ent_kind == 'permen':
                                    times, ent = get_perm_entropy(filtered, metric, measfreq, dtime, overlap)
                                elif ent_kind == 'msen':
                                    times, ent = get_ms_entropy(filtered, metric, scale, measfreq, dtime, overlap)

                            else: # middle bands
                                sos = signal.butter(8,[bands[k-1],bands[k]], btype = 'bandpass', output = 'sos', fs = measfreq)
                                filtered = signal.sosfilt(sos, data[i][key][eeg])
                                if ent_kind == 'sampen':
                                    times, ent = get_samp_entropy(filtered, metric, measfreq, dtime, overlap)
                                elif ent_kind == 'permen':
                                    times, ent = get_perm_entropy(filtered, metric, measfreq, dtime, overlap)
                                elif ent_kind == 'msen':
                                    times, ent = get_ms_entropy(filtered, metric, scale, measfreq, dtime, overlap)

                            banddata = banddata + [bandnames[k]]*len(ent)
                            ents = ents +list(ent)
                            eegs = eegs +[eeg]*len(ent)
                            sesions = sesions + [ses]*len(ent)
                            timing = timing + [key]*len(ent)
                        except:
                            pass


        ent_data_band['sesion']=sesions
        ent_data_band['entropia'] = ents
        ent_data_band['eeg'] = eegs
        ent_data_band['banda'] = banddata
        ent_data_band['fase'] = timing

        # calculating means and stds
        ent_mean = ent_data_band.groupby(['sesion', 'eeg', 'banda', 'fase']).mean().reset_index().rename({'entropia':'mean'}, axis = 1)
        ent_std = ent_data_band.groupby(['sesion', 'eeg', 'banda', 'fase']).std().reset_index().rename({'entropia':'std'}, axis = 1)

        # Evaluating the statistical difference among the 'pre' measurements among sessions
        statsPRE = pd.DataFrame() 

        pvals = []
        diffs = []
        stds = []
        eegs = []
        bands = []
        sess = []


        for i,ses  in enumerate(sesiones[1:]):
            for j,eeg in enumerate(['EEG1','EEG2']):
                for k,name in enumerate(bandnames):
                    bands.append(name)
                    eegs.append(eeg)
                    sess.append(ses)
                    mask1 = (ent_data_band['sesion']==sesiones[i]) & (ent_data_band['eeg']==eeg) & (ent_data_band['banda']==name) & (ent_data_band['fase']==phases[0])
                    mask2 = (ent_data_band['sesion']==ses) & (ent_data_band['eeg']==eeg) & (ent_data_band['banda']==name) & (ent_data_band['fase']==phases[0])
                    ttestres = stats.ttest_ind(ent_data_band.loc[mask1,'entropia'],ent_data_band.loc[mask2,'entropia'])    
                    pvals.append(ttestres.pvalue)
                    mask3 = (ent_mean['sesion']==sesiones[i]) & (ent_mean['eeg']==eeg) & (ent_mean['banda']==name) & (ent_mean['fase']==phases[0])
                    mask4 = (ent_mean['sesion']==ses) & (ent_mean['eeg']==eeg) & (ent_mean['banda']==name) & (ent_mean['fase']==phases[0])
                    try:
                        diffs.append((100*(ent_mean.loc[mask4,'mean'].values-ent_mean.loc[mask3,'mean'].values)/ent_mean.loc[mask3,'mean'].values)[0])
                        stds.append((100*np.sqrt(ent_std.loc[mask4,'std'].values**2+ent_std.loc[mask3,'std'].values)/ent_mean.loc[mask3,'mean'].values)[0])
                    except:
                        diffs.append(np.nan)
                        stds.append(np.nan)
                        

        statsPRE['sesion'] = sess
        statsPRE['eeg'] = eegs
        statsPRE['bands'] = bands
        statsPRE['pvalues'] = pvals
        statsPRE['diff'] = diffs
        statsPRE['errdiff'] = stds


        # Evaluating the statistical difference among the 'pre' and 'post' measurements in each session
        statsPREPOST = pd.DataFrame() 

        pvals = []
        diffs = []
        stds = []
        eegs = []
        bands = []
        sess = []

        for i,ses  in enumerate(sesiones):
            for j,eeg in enumerate(['EEG1','EEG2']):
                for k,name in enumerate(bandnames):
                    bands.append(name)
                    eegs.append(eeg)
                    sess.append(ses)
                    mask1 = (ent_data_band['sesion']==ses) & (ent_data_band['eeg']==eeg) & (ent_data_band['banda']==name) & (ent_data_band['fase']==phases[0])
                    mask2 = (ent_data_band['sesion']==ses) & (ent_data_band['eeg']==eeg) & (ent_data_band['banda']==name) & (ent_data_band['fase']==phases[1])
                    ttestres = stats.ttest_ind(ent_data_band.loc[mask1,'entropia'],ent_data_band.loc[mask2,'entropia'])    
                    pvals.append(ttestres.pvalue)
                    mask3 = (ent_mean['sesion']==ses) & (ent_mean['eeg']==eeg) & (ent_mean['banda']==name) & (ent_mean['fase']==phases[0])
                    mask4 = (ent_mean['sesion']==ses) & (ent_mean['eeg']==eeg) & (ent_mean['banda']==name) & (ent_mean['fase']==phases[1])
                    try:
                        diffs.append((100*(ent_mean.loc[mask4,'mean'].values-ent_mean.loc[mask3,'mean'].values)/ent_mean.loc[mask3,'mean'].values)[0])
                        stds.append((100*np.sqrt(ent_std.loc[mask4,'std'].values**2+ent_std.loc[mask3,'std'].values)/ent_mean.loc[mask3,'mean'].values)[0])
                    except:
                        diffs.append(np.nan)
                        stds.append(np.nan)

        statsPREPOST['sesion'] = sess
        statsPREPOST['eeg'] = eegs
        statsPREPOST['bands'] = bands
        statsPREPOST['pvalues'] = pvals
        statsPREPOST['diff'] = diffs
        statsPREPOST['errdiff'] = stds

        statsPRE['signif'] = statsPRE['pvalues'] < 0.05
        statsPREPOST['signif'] = statsPREPOST['pvalues'] < 0.05



        #plt.close()

        #exporting the results as excel files
        ent_data_band.to_excel(fileroot+'_ent.xlsx')
        statsPRE.to_excel(fileroot+'_'+phases[0]+'_stats.xlsx')
        statsPREPOST.to_excel(fileroot+'_'+comparison+'_stats.xlsx')

    return 

# Gráfica de la evolución de la entropía para diferentes subjects
def plotEntropyAnalisis(subjects, phaseslist, ent_kind, metrics, step, overlap):
    bandnames = ['delta', 'theta', 'alpha', 'beta', 'gamma', 'all bands']
    # we load the data
    for i,subject in enumerate(subjects):
        phases = phaseslist[i]
        comparison = phases[0]+phases[1]
        for metric in metrics:
            fileroot = 'results/entropy_'+subject+'_'+ent_kind+'_'+str(metric)+'_'+str(step)+'_'+str(overlap)
            ent = pd.read_excel(fileroot+'_ent.xlsx')
            ent['metric'] = metric
            ent['subject'] = subject
            stat = pd.read_excel(fileroot+'_'+phases[0]+'_stats.xlsx')
            stat['metric'] = metric
            stat['subject'] = subject
            stat_prepost = pd.read_excel(fileroot+'_'+comparison+'_stats.xlsx')
            stat_prepost['metric'] = metric
            stat_prepost['subject'] = subject
            try:
                ent_data = pd.concat([ent_data, ent])
                stat_data = pd.concat([stat_data, stat])
                stat_prepost_data = pd.concat([stat_prepost_data, stat_prepost])
            except:
                ent_data = ent
                stat_data = stat
                stat_prepost_data = stat_prepost
            # creating a new column with the subject + session information
            ent_data['info'] = ent_data['subject']+'_'+ent_data['sesion'].astype('str')+'_'+ent_data['fase']
            stat_data['info'] = stat_data['subject']+'_'+stat_data['sesion'].astype('str')
            stat_prepost_data['info'] = stat_prepost_data['subject']+'_'+stat_prepost_data['sesion'].astype('str')

                    
    
    # A figure will be created for each band and each EEG. In each case a figure showing the evolution of the calculated
    # entropy as a function of the used metric. The session number, or control number, will be used as the hue
    
    
    for band in bandnames:
        for eeg in ['EEG1', 'EEG2']:
            ###################################################################################
            ################## GENERATING THE PLOT
            if subjects[-1]!='control':
                fig = plt.figure(figsize=(10,16))
                spec = gridspec.GridSpec(nrows = 2, ncols=1, figure=fig)
                spec.update(left=0.05, right= 0.99, top = 0.95, bottom = 0.52, wspace=0.1, hspace = 0.25)
                spec2 = gridspec.GridSpec(nrows = 2, ncols=1, figure=fig)
                spec2.update(left=0.05, right= 0.99, top = 0.46, bottom = 0.03, wspace=0.1, hspace = 0.25)
            else:
                fig = plt.figure(figsize=(10,8))
                spec = gridspec.GridSpec(nrows = 2, ncols=1, figure=fig)
                spec.update(left=0.05, right= 0.99, top = 0.9, bottom = 0.1, wspace=0.1, hspace = 0.25)

                

            line1 = []
            line2 = []
            line3 = []
            line4 = []
            line1.append(plt.subplot(spec[0,0]))
            line2.append(plt.subplot(spec[1,0], sharey = line1[0]))
            if subjects[-1]!='control':
                line3.append(plt.subplot(spec2[0,0]))
                line4.append(plt.subplot(spec2[1,0], sharey = line3[0]))


            ax = [line1,line2,line3,line4]
  
            #ax.set_title('Evolución entropía\n modo: '+key+', señal: '+eeg)
            if ent_kind == 'sampen':
                fig.suptitle('Sample Entropy; band: '+band+'; eeg: '+eeg)
            elif ent_kind == 'permen':
                fig.suptitle('Permutation Entropy; band: '+band+'; eeg: '+eeg)

            for i, fase in enumerate(['pre', 'post']):
                mask = ((ent_data['fase']==fase) & (ent_data['eeg']==eeg) & (ent_data['banda']==band)) | ((ent_data['eeg']==eeg) & (ent_data['banda']==band) & (ent_data['subject']=='control'))
                sns.boxplot(data = ent_data[mask], x='metric', y='entropia', hue ='info', ax = ax[i][0])
                ax[i][0].legend(fontsize=7, loc = 'upper right')
                ax[i][0].set_title("Evolución Entropía "+ent_kind+' '+eeg+' '+'paso: %d; '%step + 'banda: '+ band + '; fase: '+ fase)

            subnames = '_'.join(subjects)
            if subjects[-1]!='control':                
                mask = (stat_data['eeg']==eeg) & (stat_data['bands']==band) & (stat_data['subject']!='control')
                sns.barplot(data = stat_data[mask], x='metric', y='diff', hue = 'sesion', ax = ax[2][0])
                ax[2][0].axhline(y=0)
                ax[2][0].legend(fontsize=7, loc = 'upper left')
                ax[2][0].set_title("cambio "+ subject[-1] +" PRE-PRE "+ent_kind+' '+eeg+' '+'paso: %d; '%step + 'banda: '+ band)
                #sns.boxplot(data = statsPREPOST[mask], x='bands', y='diff', hue = 'sesion', ax = ax[j][3])
                #ax[j][3].legend(fontsize=4)
                mask = (stat_prepost_data['eeg']==eeg) & (stat_prepost_data['bands']==band) & (stat_prepost_data['subject']!='control')
                sns.barplot(data = stat_prepost_data[mask], x='metric', y='diff', hue = 'sesion', ax = ax[3][0])
                ax[3][0].axhline(y=0)
                ax[3][0].legend(fontsize=7, loc = 'upper left')
                ax[3][0].set_title("cambio "+ subject[-1] +" PRE-POST "+ent_kind+' '+eeg+' '+'paso: %d; '%step + 'banda: '+ band)

            filename = 'images/entropy_'+subnames+'_'+ent_kind+'_'+eeg+'_'+band+'_'+str(overlap)+'.png'
            plt.savefig(filename, dpi = 400)
            plt.close()
            
# Gráfica de la evolución de la entropía para diferentes subjects
def plotEntropyAnalisisMS(subjects, phaseslist, metrics, scales, step, overlap):
    ent_kind = 'msen'
    bandnames = ['delta', 'theta', 'alpha', 'beta', 'gamma', 'all bands']
    # we load the data
    for i,subject in enumerate(subjects):
        phases = phaseslist[i]
        comparison = phases[0]+phases[1]
        for metric in metrics:
            for scale in scales:
                fileroot = 'results/entropy_'+subject+'_'+ent_kind+'_'+str(metric)+'_'+str(scale)+'_'+str(step)+'_'+str(overlap)
                ent = pd.read_excel(fileroot+'_ent.xlsx')
                ent['metric'] = metric
                ent['subject'] = subject
                ent['scale'] = scale
                stat = pd.read_excel(fileroot+'_'+phases[0]+'_stats.xlsx')
                stat['metric'] = metric
                stat['subject'] = subject
                stat['scale'] = scale
                stat_prepost = pd.read_excel(fileroot+'_'+comparison+'_stats.xlsx')
                stat_prepost['metric'] = metric
                stat_prepost['subject'] = subject
                stat_prepost['scale'] = scale
                try:
                    ent_data = pd.concat([ent_data, ent])
                    stat_data = pd.concat([stat_data, stat])
                    stat_prepost_data = pd.concat([stat_prepost_data, stat_prepost])
                except:
                    ent_data = ent
                    stat_data = stat
                    stat_prepost_data = stat_prepost
                # creating a new column with the subject + session information
                ent_data['info'] = ent_data['subject']+'_'+ent_data['sesion'].astype('str')+'_'+ent_data['fase']
                stat_data['info'] = stat_data['subject']+'_'+stat_data['sesion'].astype('str')
                stat_prepost_data['info'] = stat_prepost_data['subject']+'_'+stat_prepost_data['sesion'].astype('str')

                    
    
    # A figure will be created for each band, each EEG, and each metric. In each case a figure showing the evolution of the calculated
    # entropy as a function of the used metric. The session number, or control number, will be used as the hue
    
    for metric in metrics:
        for band in bandnames:
            for eeg in ['EEG1', 'EEG2']:
                ###################################################################################
                ################## GENERATING THE PLOT
                if subjects[-1]!='control':
                    fig = plt.figure(figsize=(10,16))
                    spec = gridspec.GridSpec(nrows = 2, ncols=1, figure=fig)
                    spec.update(left=0.05, right= 0.99, top = 0.95, bottom = 0.52, wspace=0.1, hspace = 0.25)
                    spec2 = gridspec.GridSpec(nrows = 2, ncols=1, figure=fig)
                    spec2.update(left=0.05, right= 0.99, top = 0.46, bottom = 0.03, wspace=0.1, hspace = 0.25)
                else:
                    fig = plt.figure(figsize=(10,8))
                    spec = gridspec.GridSpec(nrows = 2, ncols=1, figure=fig)
                    spec.update(left=0.05, right= 0.99, top = 0.9, bottom = 0.1, wspace=0.1, hspace = 0.25)



                line1 = []
                line2 = []
                line3 = []
                line4 = []
                line1.append(plt.subplot(spec[0,0]))
                line2.append(plt.subplot(spec[1,0], sharey = line1[0]))
                if subjects[-1]!='control':
                    line3.append(plt.subplot(spec2[0,0]))
                    line4.append(plt.subplot(spec2[1,0], sharey = line3[0]))


                ax = [line1,line2,line3,line4]

                #ax.set_title('Evolución entropía\n modo: '+key+', señal: '+eeg)
                if ent_kind == 'sampen':
                    fig.suptitle('Sample Entropy; band: '+band+'; eeg: '+eeg+'; metric: '+ metric)
                elif ent_kind == 'permen':
                    fig.suptitle('Permutation Entropy; band: '+band+'; eeg: '+eeg+'; metric: '+ metric)

                for i, fase in enumerate(['pre', 'post']):
                    mask = ((ent_data['fase']==fase) & (ent_data['eeg']==eeg) & (ent_data['banda']==band) & (ent_data['metric']==metric)) | ((ent_data['metric']==metric) & (ent_data['eeg']==eeg) & (ent_data['banda']==band) & (ent_data['subject']=='control'))
                    sns.boxplot(data = ent_data[mask], x='scale', y='entropia', hue ='info', ax = ax[i][0])
                    ax[i][0].legend(fontsize=7, loc = 'upper right')
                    ax[i][0].set_title("Evolución Entropía "+ent_kind+' '+eeg+' '+'paso: %d; '%step + 'banda: '+ band + '; fase: '+ fase)

                subnames = '_'.join(subjects)
                if subjects[-1]!='control':                
                    mask = (stat_data['eeg']==eeg) & (stat_data['bands']==band) & (stat_data['metric']==metric) & (stat_data['subject']!='control')
                    sns.barplot(data = stat_data[mask], x='scale', y='diff', hue = 'sesion', ax = ax[2][0])
                    ax[2][0].axhline(y=0)
                    ax[2][0].legend(fontsize=7, loc = 'upper left')
                    ax[2][0].set_title("cambio "+ subject[-1] +" PRE-PRE "+ent_kind+' '+eeg+' '+'paso: %d; '%step + 'banda: '+ band)
                    #sns.boxplot(data = statsPREPOST[mask], x='bands', y='diff', hue = 'sesion', ax = ax[j][3])
                    #ax[j][3].legend(fontsize=4)
                    mask = (stat_prepost_data['eeg']==eeg) & (stat_prepost_data['bands']==band) & (stat_prepost_data['subject']!='control') & (stat_prepost_data['metric']==metric)
                    sns.barplot(data = stat_prepost_data[mask], x='scale', y='diff', hue = 'sesion', ax = ax[3][0])
                    ax[3][0].axhline(y=0)
                    ax[3][0].legend(fontsize=7, loc = 'upper left')
                    ax[3][0].set_title("cambio "+ subject[-1] +" PRE-POST "+ent_kind+' '+eeg+' '+'paso: %d; '%step + 'banda: '+ band)

                filename = 'images/entropy_'+subnames+'_'+ent_kind+'_'+eeg+'_'+band+'_'+str(metric)+'_'+str(overlap)+'.png'
                plt.savefig(filename, dpi = 400)
                plt.close()

# Datos de Control

In [6]:
datosC = []
controles = np.array([1,2,3])

for con in controles:
    datos = {}
    # inicial
    try:
        datos['inicial'] = pd.read_csv('../data/Control/C%d_inicial.txt'%con, index_col=False, header=6)
    except:
        datos['inicial'] = pd.DataFrame(columns=['EEG1','EEG2','EMG','ECG'])
    # final
    try:
        datos['final'] = pd.read_csv('../data/Control/C%d_final.txt'%con, index_col=False, header=6)
    except:
        datos['final'] = pd.DataFrame(columns=['EEG1','EEG2','EMG','ECG'])
        
    datosC.append(datos)


## Cálculo de Entropía

In [4]:
# for sample and permutation entropies
for metric in np.arange(2,20,2):
        getEntropyAnalysis('control', ['inicial', 'final'], datosC,'sampen', controles, metric, 1, 240, 1, 0.8)
for metric in np.arange(3,20,2):
        getEntropyAnalysis('control', ['inicial', 'final'], datosC,'permen', controles, metric, 1, 240, 1, 0.8)
        
# now for the multiescale entropy
for scale in np.arange(2,20,2):
        getEntropyAnalysis('control', ['inicial', 'final'], datosC,'msen', controles, 2, scale, 240, 1, 0.8)
        getEntropyAnalysis('control', ['inicial', 'final'], datosC,'msen', controles, 3, scale, 240, 1, 0.8)

# Datos Paciente 1

In [5]:
datosP1 = []
sesiones1 = np.array([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])

for ses in sesiones1:
    datos = {}
    # presesion
    try:
        datos['pre'] = pd.read_csv('../data/P1/P1_pre_sesion_%d.txt'%ses, index_col=False, header=6)
    except:
        datos['pre'] = pd.DataFrame(columns=['EEG1','EEG2','EMG','ECG'])
    # sesion
    try:
        datos['sesion'] = pd.read_csv('../data/P1/P1_sesion_%d.txt'%ses, index_col=False, header=6)
    except:
        datos['sesion'] = pd.DataFrame(columns=['EEG1','EEG2','EMG','ECG'])
        
    try:
        datos['post'] = pd.read_csv('../data/P1/P1_post_sesion_%d.txt'%ses, index_col=False, header=6)
    except:
        datos['post'] = pd.DataFrame(columns=['EEG1','EEG2','EMG','ECG'])
        
    datosP1.append(datos)

In [6]:
# for sample and permutation entropies
for metric in np.arange(2,20,2):
        getEntropyAnalysis('paciente1', ['pre', 'post'], datosP1,'sampen', sesiones1, metric, 1, 240, 1, 0.8)
for metric in np.arange(3,20,2):
        getEntropyAnalysis('paciente1', ['pre', 'post'], datosP1,'permen', sesiones1, metric, 1, 240, 1, 0.8)
        
# now for the multiescale entropy
for scale in np.arange(2,20,2):
        getEntropyAnalysis('paciente1', ['pre', 'post'], datosP1,'msen', sesiones1, 2, scale, 240, 1, 0.8)
        getEntropyAnalysis('paciente1', ['pre', 'post'], datosP1,'msen', sesiones1, 3, scale, 240, 1, 0.8)

# Datos Paciente 2

In [7]:
datosP2 = []
sesiones2 = np.array([1,2,3,4])

for ses in sesiones2:
    datos = {}
    # presesion
    try:
        datos['pre'] = pd.read_csv('../data/P2/P2_pre_sesion_%d.txt'%ses, index_col=False, header=6)
    except:
        datos['pre'] = pd.DataFrame(columns=['EEG1','EEG2','EMG','ECG'])
        
    try:
        datos['post'] = pd.read_csv('../data/P2/P2_post_sesion_%d.txt'%ses, index_col=False, header=6)
    except:
        datos['post'] = pd.DataFrame(columns=['EEG1','EEG2','EMG','ECG'])
        
    datosP2.append(datos)

In [8]:
# for sample and permutation entropies
for metric in np.arange(2,20,2):
        getEntropyAnalysis('paciente2', ['pre', 'post'], datosP2,'sampen', sesiones2, metric, 1, 240, 1, 0.8)
for metric in np.arange(3,20,2):
        getEntropyAnalysis('paciente2', ['pre', 'post'], datosP2,'permen', sesiones2, metric, 1, 240, 1, 0.8)
        
# now for the multiescale entropy
for scale in np.arange(2,20,2):
        getEntropyAnalysis('paciente2', ['pre', 'post'], datosP2,'msen', sesiones2, 2, scale, 240, 1, 0.8)
        getEntropyAnalysis('paciente2', ['pre', 'post'], datosP2,'msen', sesiones2, 3, scale, 240, 1, 0.8)

# Datos Paciente 3

In [9]:
datosP3 = []
sesiones3 = np.array([1,2,3,4,5])

for ses in sesiones3:
    datos = {}
    # presesion
    try:
        datos['pre'] = pd.read_csv('../data/P3/P3_pre_sesion_%d.txt'%ses, index_col=False, header=6)
    except:
        datos['pre'] = pd.DataFrame(columns=['EEG1','EEG2','EMG','ECG'])
        
    try:
        datos['post'] = pd.read_csv('../data/P3/P3_post_sesion_%d.txt'%ses, index_col=False, header=6)
    except:
        datos['post'] = pd.DataFrame(columns=['EEG1','EEG2','EMG','ECG'])
        
    datosP3.append(datos)

In [10]:
# for sample and permutation entropies
for metric in np.arange(2,20,2):
        getEntropyAnalysis('paciente3', ['pre', 'post'], datosP3,'sampen', sesiones3, metric, 1, 240, 1, 0.8)
for metric in np.arange(3,20,2):
        getEntropyAnalysis('paciente3', ['pre', 'post'], datosP3,'permen', sesiones3, metric, 1, 240, 1, 0.8)
        
# now for the multiescale entropy
for scale in np.arange(2,20,2):
        getEntropyAnalysis('paciente3', ['pre', 'post'], datosP3,'msen', sesiones3, 2, scale, 240, 1, 0.8)
        getEntropyAnalysis('paciente3', ['pre', 'post'], datosP3,'msen', sesiones3, 3, scale, 240, 1, 0.8)

# Plotting

## Control Spectrograms

In [9]:
createSpectralFigureBands('control', datosC, 0, controles, 'inicial', 'EEG1', 3, 240, 120, 1, 0.8)
createSpectralFigureBands('control', datosC, 0, controles, 'final', 'EEG1', 3, 240, 120, 1, 0.8)
createSpectralFigureBands('control', datosC, 1, controles, 'inicial', 'EEG1', 3, 240, 120, 1, 0.8)
createSpectralFigureBands('control', datosC, 2, controles, 'inicial', 'EEG1', 3, 240, 120, 1, 0.8)

In [3]:
createSpectralFigureBands('control', datosC, 0, controles, 'inicial', 'EEG2', 3, 240, 120, 1, 0.8)
createSpectralFigureBands('control', datosC, 0, controles, 'final', 'EEG2', 3, 240, 120, 1, 0.8)
createSpectralFigureBands('control', datosC, 1, controles, 'inicial', 'EEG2', 3, 240, 120, 1, 0.8)
createSpectralFigureBands('control', datosC, 2, controles, 'inicial', 'EEG2', 3, 240, 120, 1, 0.8)

NameError: name 'createSpectralFigureBands' is not defined

## Control evolution

In [23]:
plotEntropyAnalisis(['control'],[['inicial','final']],'sampen',np.arange(2,20,2),1,0.8)
plotEntropyAnalisis(['control'],[['inicial','final']],'permen',np.arange(3,20,2),1,0.8)

U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib

U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4012: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\migue

In [24]:
plotEntropyAnalisisMS(['control'],[['inicial','final']], [2,3], np.arange(2,20,2), 1, 0.8)

## Paciente 1 Evolución

In [25]:
plotEntropyAnalisis(['control', 'paciente1'],[['inicial','final'],['pre','post']],'sampen',np.arange(2,20,2),1,0.8)
plotEntropyAnalisis(['control', 'paciente1'],[['inicial','final'],['pre','post']],'permen',np.arange(3,20,2),1,0.8)


U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib

U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4011: RuntimeWarning: invalid value encountered in multiply
  lerp_interpolation = asanyarray(add(a, diff_b_a*t, out=out))
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\

In [26]:
plotEntropyAnalisisMS(['control', 'paciente1'],[['inicial','final'],['pre','post']], [2,3], np.arange(2,20,2), 1, 0.8)

## Paciente 2 Evolución

In [27]:
plotEntropyAnalisis(['control', 'paciente2'],[['inicial','final'],['pre','post']],'sampen',np.arange(2,20,2),1,0.8)
plotEntropyAnalisis(['control', 'paciente2'],[['inicial','final'],['pre','post']],'permen',np.arange(3,20,2),1,0.8)
plotEntropyAnalisisMS(['control', 'paciente2'],[['inicial','final'],['pre','post']], [2,3], np.arange(2,20,2), 1, 0.8)

U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4012: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\migue

U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4012: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\migue

## Paciente 3 Evolución

In [28]:
plotEntropyAnalisis(['control', 'paciente3'],[['inicial','final'],['pre','post']],'sampen',np.arange(2,20,2),1,0.8)
plotEntropyAnalisis(['control', 'paciente3'],[['inicial','final'],['pre','post']],'permen',np.arange(3,20,2),1,0.8)
plotEntropyAnalisisMS(['control', 'paciente3'],[['inicial','final'],['pre','post']], [2,3], np.arange(2,20,2), 1, 0.8)

U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib

U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4012: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\migue

### Paciente 4 Evolución

In [29]:
plotEntropyAnalisis(['control', 'paciente4'],[['inicial','final'],['pre','post']],'sampen',np.arange(2,20,2),1,0.8)
plotEntropyAnalisis(['control', 'paciente4'],[['inicial','final'],['pre','post']],'permen',np.arange(3,20,2),1,0.8)
plotEntropyAnalisisMS(['control', 'paciente4'],[['inicial','final'],['pre','post']], [2,3], np.arange(2,20,2), 1, 0.8)

U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib

U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4012: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\migue

### Paciente 5 Evolución

In [30]:
plotEntropyAnalisis(['control', 'paciente5'],[['inicial','final'],['pre','post']],'sampen',np.arange(2,20,2),1,0.8)
plotEntropyAnalisis(['control', 'paciente5'],[['inicial','final'],['pre','post']],'permen',np.arange(3,20,2),1,0.8)
plotEntropyAnalisisMS(['control', 'paciente5'],[['inicial','final'],['pre','post']], [2,3], np.arange(2,20,2), 1, 0.8)

U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib

U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib

### Paciente 6 Evolución

In [31]:
plotEntropyAnalisis(['control', 'paciente6'],[['inicial','final'],['pre','post']],'sampen',np.arange(2,20,2),1,0.8)
plotEntropyAnalisis(['control', 'paciente6'],[['inicial','final'],['pre','post']],'permen',np.arange(3,20,2),1,0.8)
plotEntropyAnalisisMS(['control', 'paciente6'],[['inicial','final'],['pre','post']], [2,3], np.arange(2,20,2), 1, 0.8)

U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib

U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4012: RuntimeWarning: invalid value encountered in subtract
  subtract(b, diff_b_a * (1 - t), out=lerp_interpolation, where=t>=0.5)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\miguel\pyenvs\entropy\lib\site-packages\numpy\lib\function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
U:\migue